>[Introduction](#scrollTo=1vNpobIiIcYq)

>[Loading the data](#scrollTo=2lCpzgnrIgxv)

>>[Task 1: Randomize the data](#scrollTo=MqeZPcDyIl6P)

>[Defining the functions](#scrollTo=u49aNgQLIr36)

>>[Task 2: Complete the functions](#scrollTo=vv0kJy_DIwYi)

>[Creating training and test sets](#scrollTo=MQ6GkWVHI0-O)

>>[Task 3: Complete the codes](#scrollTo=uPhdEtlfI5fi)

>[Defining the input functions](#scrollTo=voToN4G2JBaa)

>[Defining the train_model functions](#scrollTo=EM44SV9nJFcf)

>>[Task 4: Complete the code](#scrollTo=dp1Bmsu_JJGn)

>[Traing and predict](#scrollTo=nA2rvcPYJQ33)

>>[Task 5: Get a loss less than 8.7](#scrollTo=DWJVDOiJJUmB)



# Introduction

In this task, we'll be using the Boston housing dataset that comes prebuilt with sklearn. The first part is usual improting routine.

In [0]:
from __future__ import print_function

import math

from IPython import display
from matplotlib import cm
from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import metrics
import tensorflow as tf
from tensorflow.python.data import Dataset

tf.logging.set_verbosity(tf.logging.ERROR)
pd.options.display.max_rows = 10
pd.options.display.float_format = '{:.1f}'.format

# Loading the data

Here we load the dataset. **It is important you read the description and know which column denotes what since the names are not meaningful by themselves**

In [0]:
from sklearn.datasets import load_boston

boston = load_boston()
print(boston.DESCR)

As you can see, the boston dataset has a different format than what we have been using, so we will convert it to a Pandas dataframe.

In [0]:
boston_housing_dataframe = pd.DataFrame(boston.data)
boston_housing_dataframe.columns = boston.feature_names
boston_housing_dataframe['PRICE'] = boston.target

Check the data

In [0]:
boston_housing_dataframe.describe()

## Task 1: Randomize the data

Your first task is to write the code to randomize the data.

In [0]:
# RANDOMIZE THE DATA

# Defining the functions

## Task 2: Complete the functions

For our target, we'll be using **"PRICE"** column and for the input features, we'll be using all the remaining columns.

Your tasks are - 

1. Complete the **selected_features** list to include the rest of the input features.
2. complete the **processed_features** function to construct a dataframe and extract the PRICE column

In [0]:
def preprocess_features(boston_housing_dataframe):

  selected_features = boston_housing_dataframe[
    ["CRIM",
     "ZN",
     "INDUS",
     "CHAS",
     "NOX",
     # WRITE THE REST OF THE FEATURE COLUMNS
    ]]
  processed_features = selected_features.copy()
  return processed_features

def preprocess_targets(boston_housing_dataframe):
  output_targets = # CONSTRUCT A DATAFRAME HERE WHICH HOLDS THE PRICE COLUMN FROM THE ACTUAL DATAFRAME
  return output_targets

# Creating training and test sets

## Task 3: Complete the codes

There are only 506 datapoints. We'll use the first 380 examples (~75%) for training data and last 126 for validation.

Your task is to create the appropriate datasets.

In [0]:
training_examples = # GET THE FIRST 380 FEATURE VALUES AND PREPROCESS THEM
training_examples.describe()

In [0]:
training_targets = # GET THE LAST 126 PRICE VALUES AND PREPROCESS IT
training_targets.describe()

In [0]:
validation_examples = # GET THE LAST 126 FEATURE VALUES AND PREPROCESS THEM
validation_examples.describe()

In [0]:
validation_targets = # GET THE LAST 126 PRICE VALUES AND PREPROCESS IT
validation_targets.describe()

# Defining the input functions

The input function and construct_feature_column function is defined as before.

In [0]:
def my_input_fn(features, targets, batch_size=1, shuffle=True, num_epochs=None):
   
    
    features = {key:np.array(value) for key,value in dict(features).items()}                                           
 
    ds = Dataset.from_tensor_slices((features,targets)) 
    ds = ds.batch(batch_size).repeat(num_epochs)

    if shuffle:
      ds = ds.shuffle(10000)

    features, labels = ds.make_one_shot_iterator().get_next()
    return features, labels

In [0]:
def construct_feature_columns(input_features):

  return set([tf.feature_column.numeric_column(my_feature)
              for my_feature in input_features])

# Defining the train_model functions

## Task 4: Complete the code

The train model function is all the same. Your task is to - 

1. Put the num_epochs and shuffle parameters accordingly (they're all prediction functions, so they'll have a certain value)
2. Predict on the validation set and convert into numpy array

In [0]:
def train_model(
    learning_rate,
    steps,
    batch_size,
    training_examples,
    training_targets,
    validation_examples,
    validation_targets):

  periods = 10
  steps_per_period = steps / periods

  my_optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
  my_optimizer = tf.contrib.estimator.clip_gradients_by_norm(my_optimizer, 5.0)
  linear_regressor = tf.estimator.LinearRegressor(
      feature_columns=construct_feature_columns(training_examples),
      optimizer=my_optimizer
  )
  
  training_input_fn = lambda: my_input_fn(
      training_examples, 
      training_targets["PRICE"], 
      batch_size=batch_size)
  predict_training_input_fn = lambda: my_input_fn(
      training_examples, 
      training_targets["PRICE"], 
      # PUT num_epochs AND shuffle VALUES ACCORDINGLY
  )
  predict_validation_input_fn = lambda: my_input_fn(
      validation_examples, validation_targets["PRICE"], 
      # PUT num_epochs AND shuffle VALUES ACCORDINGLY
  )

  print("Training model...")
  print("RMSE (on training data):")
  training_rmse = []
  validation_rmse = []
  for period in range (0, periods):
    linear_regressor.train(
        input_fn=training_input_fn,
        steps=steps_per_period,
    )
 
    training_predictions = linear_regressor.predict(input_fn=predict_training_input_fn)
    training_predictions = np.array([item['predictions'][0] for item in training_predictions])
    
    validation_predictions = # MAKE THE PREDICTION ON VALIDATION SET
    validation_predictions = # CONVERT INTO A NUMPY ARRAY
    
    training_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(training_predictions, training_targets))
    validation_root_mean_squared_error = math.sqrt(
        metrics.mean_squared_error(validation_predictions, validation_targets))
    print("  period %02d : %0.2f" % (period, training_root_mean_squared_error))
    training_rmse.append(training_root_mean_squared_error)
    validation_rmse.append(validation_root_mean_squared_error)
  print("Model training finished.")

  plt.ylabel("RMSE")
  plt.xlabel("Periods")
  plt.title("Root Mean Squared Error vs. Periods")
  plt.tight_layout()
  plt.plot(training_rmse, label="training")
  plt.plot(validation_rmse, label="validation")
  plt.legend()

  return linear_regressor

# Traing and predict

## Task 5: Get a loss less than 8.7

Finally, find a proper learning rate, steps and batch size to get a loss less than 8.7

In [0]:
linear_regressor = train_model(
    learning_rate= # PUT A VALUE,
    steps=# PUT A VALUE,
    batch_size=# PUT A VALUE,
    training_examples=training_examples,
    training_targets=training_targets,
    validation_examples=validation_examples,
    validation_targets=validation_targets)